In [1]:
from datascience import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import json

In [2]:
# 10/08 Note: research_fund_comb.csv is a complete csv of the initial pdf file 
combined_t = Table.read_table('data/research_fund_comb.csv')
combined_t.show(20)

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati..,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc..,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda..,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit..,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage..,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc.","$73,942",nan,nan


In [3]:
# 10/09 Note: All sponsors downloaded from the Cal Phoebe site 
phoebe = Table.read_table("data/PhoebeSearchResults.csv")
phoebe

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym
Federal,Other Federal Category,All Other Federal,150,CIA Central Intelligence Agency,CIA
Federal,Other Federal Category,All Other Federal,5510,Federal Mediation & Conciliation Service,FMCS
Federal,Other Federal Category,All Other Federal,5440,National Commission on Air Quality,nan
Federal,Other Federal Category,All Other Federal,5350,VA Department of Veterans Affairs,VA
Federal,Other Federal Category,All Other Federal,5325,Smithsonian - National Museum Natural History,nan
Federal,Other Federal Category,All Other Federal,5305,Smithsonian - Office of Fellowships & Grants,nan
Federal,Other Federal Category,All Other Federal,5302,Smithsonian - Office of Environmental Sciences,nan
Federal,Other Federal Category,All Other Federal,5301,Smithsonian Institution,nan
Federal,Other Federal Category,All Other Federal,5290,National Archives and Records Administration,nan
Federal,Other Federal Category,All Other Federal,5285,SBA Small Business Administration,SBA


In [4]:
phoebe_trunc = phoebe.with_column("SponsorName_trunc", 
                                 phoebe.apply(lambda x: x[0:28], "Sponsor"))
phoebe_trunc

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym,SponsorName_trunc
Federal,Other Federal Category,All Other Federal,150,CIA Central Intelligence Agency,CIA,CIA Central Intelligence Age
Federal,Other Federal Category,All Other Federal,5510,Federal Mediation & Conciliation Service,FMCS,Federal Mediation & Concilia
Federal,Other Federal Category,All Other Federal,5440,National Commission on Air Quality,nan,National Commission on Air Q
Federal,Other Federal Category,All Other Federal,5350,VA Department of Veterans Affairs,VA,VA Department of Veterans Af
Federal,Other Federal Category,All Other Federal,5325,Smithsonian - National Museum Natural History,nan,Smithsonian - National Museu
Federal,Other Federal Category,All Other Federal,5305,Smithsonian - Office of Fellowships & Grants,nan,Smithsonian - Office of Fell
Federal,Other Federal Category,All Other Federal,5302,Smithsonian - Office of Environmental Sciences,nan,Smithsonian - Office of Envi
Federal,Other Federal Category,All Other Federal,5301,Smithsonian Institution,nan,Smithsonian Institution
Federal,Other Federal Category,All Other Federal,5290,National Archives and Records Administration,nan,National Archives and Record
Federal,Other Federal Category,All Other Federal,5285,SBA Small Business Administration,SBA,SBA Small Business Administr


In [5]:
combined_clean_name = combined_t.with_column('Direct Sponsor', 
                                        combined_t.apply(lambda x: x.split(".")[0], 'Direct Sponsor'))
combined_clean_name

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc","$73,942",nan,nan


In [8]:
# combined_clean_name.join("Direct Sponsor", phoebe_trunc, "SponsorName_trunc")
# 10/09 Note: Finding the corresponding "complete" names of sponsors
extended_names = {}
remnants = {}
for i in range(combined_clean_name.num_rows):
    source_name = combined_clean_name.column('Direct Sponsor')[i]
    target = phoebe_trunc.column("Sponsor")
    for j in range(len(target)):
        target_name = target[j]
        if (i) not in extended_names.keys() and source_name in target_name:
            extended_names[(i)] = target_name

In [9]:
extended_names

{0: 'LBNL Lawrence Berkeley National Laboratory',
 1: 'SAMHSA Center for Substance Abuse Prevention',
 2: 'UC Office of the President',
 3: 'Andrew W Mellon Foundation',
 4: 'LYRASIS',
 5: 'NSF National Science Foundation',
 6: 'Middlebury College',
 7: 'Bay Areas Urban Area Security Initiative',
 8: 'California Emergency Management Agency',
 9: 'Abt Associates, Inc.',
 10: 'California Department of Forestry and Fire Protection',
 11: 'NIH Office of the Director',
 12: 'Oregon, State of',
 13: 'Research Triangle Institute',
 14: 'UC Davis',
 15: 'UC San Francisco',
 16: 'USDA Forest Service',
 17: 'California Department of Industrial Relations',
 18: 'California Department of Water Resources',
 19: 'California Dried Plum Board',
 20: 'California Energy Commission',
 21: 'California Polytechnic State University Foundation',
 22: 'California Public Utilities Commission',
 23: 'Cotton Inc.',
 24: 'California Department of Resources Recycling and Recovery (DO NOT USE - USE 00F428 Instead)'

In [16]:
combined_t.where('Direct Sponsor', 'DO NOT USE (Use 00VA27) ..')

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Vice Chancellor - Research,Academic Research Units,IIS Institute of International Studies,DO NOT USE (Use 00VA27) ..,"$1,800,000",$0,nan
Vice Chancellor - Research,VC Res Other Research Units,Center of Evaluation for Global Action,DO NOT USE (Use 00VA27) ..,"$1,344,000","$600,000",nan


In [11]:
orig = (np.array(list(extended_names.keys())))
missing = []
for elem in range(combined_clean_name.num_rows):
    if elem not in orig:
        missing += [elem]
print("Row IDs of sponsors in combined_clean_name not found", missing)

Row IDs of sponsors in combined_clean_name not found [1434, 2849]


In [12]:
combined_clean_name = combined_clean_name.with_column('index', range(combined_clean_name.num_rows))
combined_clean_name.show(3)

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,index
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan,0
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan,1
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan,2


In [13]:
combined_clean_name.row([1434, 2849])

Row(﻿Control Unit=array(['L&S College of Letters & Sciences', 'Vice Chancellor - Research'], 
      dtype='<U40'), Division=array(['L&S Biological Sciences', 'Academic Research Units'], 
      dtype='<U47'), Department=array(['MCB Molecular & Cell Biology', 'Institute of European Studies'], 
      dtype='<U60'), Direct Sponsor=array(['Helmholtz Zentrum MÃ¼nchen', 'Stifterverband für die Deutsch'], 
      dtype='<U33'), 2014=array(['nan', '$0 '], 
      dtype='<U13'), 2015=array(['nan', '$0 '], 
      dtype='<U12'), 2016=array(['$0 ', '$0 '], 
      dtype='<U32'), index=array([1434, 2849]))

In [14]:
#adding the leftover 

# 10/09 Note: These two names are found in the corresponding rows of combined_clean_name but were either mispelled 
# or warped 
extended_names[1434] = "Helmholtz Zentrum Munchen"
extended_names[2849] = "Stifterverband fur die Deutsch"

In [15]:
len(extended_names) 

2872

## Starting with extended_names (as a means to join phoebe and data)

In [16]:
extended_names_tbl = Table().with_columns(['index', extended_names.keys(),
                                           'Sponsor', extended_names.values()])
extended_names_tbl.show(2)

index,Sponsor
0,LBNL Lawrence Berkeley National Laboratory
1,SAMHSA Center for Substance Abuse Prevention


In [17]:
combined_clean_name = combined_clean_name.join('index', extended_names_tbl)
combined_clean_name.show(100)

index,﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,Sponsor
0,Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan,LBNL Lawrence Berkeley National Laboratory
1,Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan,SAMHSA Center for Substance Abuse Prevention
2,Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan,UC Office of the President
3,Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan,Andrew W Mellon Foundation
4,Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854",LYRASIS
5,Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda,nan,nan,"$399,446",NSF National Science Foundation
6,Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan,Middlebury College
7,Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit,"$327,226","$30,145",nan,Bay Areas Urban Area Security Initiative
8,Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage,nan,nan,"$573,828",California Emergency Management Agency
9,Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc","$73,942",nan,nan,"Abt Associates, Inc."


In [18]:
phoebe.group('Sponsor').where('count', are.above(1))

Sponsor,count
"Abt Associates, Inc.",2
Academy of Sciences of the Czech Republic,2
"Intuitive Surgical, Inc.",2
Nazarbayev University,2
Santa Clara Valley Water District,2
Spanish National Research Council,2
Twist Bioscience,2
University of London,2
Van Strum Foundation,2
World Vision Inc.,2


In [19]:
sponsor_to_class = {
    'Abt Associates, Inc.': 'Industry',
    'Academy of Sciences of the Czech Republic': 'Non Federal Governmental',
    'Intuitive Surgical, Inc.': 'Industry',
    'Nazarbayev University': 'Not for Profit',
    'Santa Clara Valley Water District': 'Non Federal Governmental',
    'Spanish National Research Council': 'Non Federal Governmental',
    'Twist Bioscience': 'Industry',
    'University of London': 'Not for Profit',
    'Van Strum Foundation': 'Not for Profit',
    'World Vision Inc.': 'Not for Profit',
    'New Profit, Inc.': 'Not for Profit',
    'Oak Ridge National Laboratory': 'Federal',
    'Shoo the Flu, LLC': 'Non Federal Governmental',
    'DO NOT USE (Use 00VA27) Wellspring Advisors, LLC' : 'Industry'
}
sponsor_to_sponsor = {
    'DO NOT USE (Use 00VA27) Wellspring Advisors, LLC' : 'Wellspring Advisors, LLC'
}
department_to_dept = {
    'Center of Evaluation for Global Action': 'Center for Effective Global Action'
}

In [17]:
def sponsor_to_class_mapper(row):
    if row.Sponsor in sponsor_to_class:
        return sponsor_to_class[row.Sponsor]
    else:
        return row['Sponsor Class']
# added 
def sponsor_to_sponsor_mapper(row):
    if row.Sponsor in sponsor_to_sponsor:
        return sponsor_to_sponsor[row.Sponsor]
    else:
        return row['Sponsor']
    
def dept_to_dept_mapper(row):
    if row.Department in department_to_dept:
        return department_to_dept[row.Department]
    else:
        return row['Department']

In [21]:
phoebe = phoebe.to_df().assign(class_clean = phoebe.to_df().apply(sponsor_to_class_mapper, axis=1))

In [22]:
phoebe = phoebe.drop_duplicates(subset='Sponsor')

In [ ]:
# phoebe = phoebe.assign(department_clean = phoebe.apply(dept_to_dept_mapper, axis=1))

In [ ]:
# uncomment if necessary 
# phoebe = phoebe.assign(sponsor_clean = phoebe.apply(sponsor_to_sponsor_mapper, axis=1))

In [23]:
phoebe = Table().from_df(phoebe)

In [24]:
combined_phoebe = combined_clean_name.join('Sponsor', phoebe)
combined_phoebe.show(5)

Sponsor,index,﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor Acronym,class_clean
"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",1471,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,NIH National Center for Rese,nan,$0,nan,Federal,DHHS,DHHS-NIH,3417,NCRR,Federal
3M Corporation,559,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3M Corporation,nan,"$250,000",nan,Industry,USA Industry Category,USA Industry,1003,3M,Industry
"4D Molecular Therapeutics, LLC",2499,Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),"4D Molecular Therapeutics, L",nan,nan,"$22,408",Industry,USA Industry Category,USA Industry,00VC9H,nan,Industry
"AFINGEN, Inc.",2502,Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),"AFINGEN, Inc",nan,nan,"$5,000",Industry,USA Industry Category,USA Industry,00VBJK,nan,Industry
"APATH, L.C.C.",1391,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,"APATH, L",$0,nan,nan,Industry,USA Industry Category,USA Industry,4928,nan,Industry


In [31]:
combined_phoebe_conc = combined_phoebe.select(['Sponsor', '\ufeffControl Unit', 'Division', 'Department', 'class_clean', '2014', '2015', '2016'])
combined_phoebe_conc.show(3)
# combined_phoebe.labels

Sponsor,﻿Control Unit,Division,Department,class_clean,2014,2015,2016
"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,nan,$0,nan
3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,nan,"$250,000",nan
"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,nan,nan,"$22,408"


In [32]:
# 10/09 Note: Beginning to  clean dollar amounts
combined_phoebe_conc_pd = combined_phoebe_conc.to_df().fillna("Null")

In [33]:
# dealing with parentheses and dollar signs 
def money_parse(strval):
    if strval == "Null" or strval == 'nan':
        return 0
    if "(" in strval:
        # Parentheses signify negatives 
        parsed = eval(strval.replace('($', '').replace(')', '').replace(',', '')) * -1
    else:
        parsed = eval(strval.replace('$', '').replace(',', ''))
    return parsed 

combined_phoebe_conc_pd['2014'] = combined_phoebe_conc_pd['2014'].apply(money_parse)
combined_phoebe_conc_pd['2015'] = combined_phoebe_conc_pd['2015'].apply(money_parse)
combined_phoebe_conc_pd['2016'] = combined_phoebe_conc_pd['2016'].apply(money_parse)

In [34]:
combined_phoebe_conc_pd.head(3)

,Sponsor,﻿Control Unit,Division,Department,class_clean,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Bios...,Industry,0,0,22408


In [35]:
# 10/09 Note: contains cleaned data and considers negatives 
combined_phoebe_conc_pd.to_csv('data/research_funding_clean.csv')

### Non-stem/stem

In [23]:
test = Table().read_table('data/research_funding_clean.csv')
test.relabel('class_clean', 'Sponsor Class')
test = test.with_column('Total', 
                       test.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))

In [ ]:
# Divisions
stem = ['CNR College of Natural Resources', 'Chemistry Department', 'College of Chemistry',
       'College of Engineering', 'College of Environmental Design', 'Division of Physical Sciences',
       'L&S Biological Sciences', 'School of Information', 'School of Optometry', 
       ]
other = ['Strategic Academic and Facilities Planning', 'UC Library', 
        'Vice Provost Academic Affairs & Faculty Welfare', 'Vice Chancellor - Student Affairs Division',
        'Vice Chancellor - Real Estate', 'Vice Chancellor - Administration', 
        'VP Academic Planning & Fac', 'Office of Equity & Inclusion',
         'EVCP Executive Vice Chancellor & Provost']

In [32]:
industry = test.where('Sponsor Class', 'Industry')
# industry = industry.where(industry.column('Total') != 0)

In [30]:
# if division or department includes sciences, chemistry, engineering, transportation, 
def non_stem(div, dept):
    keywords = ['social sciences', 'international studies', 'education', 'labor and employment',
               'law', 'haas', 'regional development', 'humanities']
    div_ = np.any([x in div.lower() for x in keywords])
    dept_ = np.any([x in dept.lower() for x in keywords])
    return div_ or dept_

In [33]:
stem_industry = industry.with_column('Non_Stem',
                                    industry.apply(non_stem, ['Division', 'Department']))
stem_industry.where('Non_Stem', True).num_rows

17

In [34]:
stem_industry.where('Non_Stem', True).sort('Total')

Unnamed: 0,Sponsor,﻿Control Unit,Division,Department,Sponsor Class,2014,2015,2016,Total,Non_Stem
1408,"Lumos Labs, Inc.",L&S College of Letters & Sciences,Division of Social Sciences,Psychology,Industry,0,0,0,0,True
1812,National Opinion Research Center,L&S College of Letters & Sciences,Division of Social Sciences,Sociology,Industry,0,0,0,0,True
1871,Oakland Raiders,L&S College of Letters & Sciences,Division of Social Sciences,The Social Science Matrix,Industry,0,0,0,0,True
1911,Pacific Gas and Electric Company,Vice Chancellor - Research,Academic Research Units,IRLE Institute for Research on Labor and Employment,Industry,0,0,0,0,True
2284,The Yield Book Inc.,Schools,Haas School of Business (Division),Null,Industry,0,0,0,0,True
180,Arup North America Ltd,Vice Chancellor - Research,Academic Research Units,IURD Institute of Urban & Regional Development,Industry,0,4000,0,4000,True
1908,Pacific Gas and Electric Company,Schools,Haas School of Business (Division),Haas Centers and Institutes,Industry,0,25859,11750,37609,True
967,Foundation Banque De France,L&S College of Letters & Sciences,Division of Social Sciences,Economics,Industry,41108,0,0,41108,True
937,Fidelity Investments,Schools,Goldman School of Public Policy,Center for Studies in Higher Education,Industry,50000,0,0,50000,True
2820,Wellintel,L&S College of Letters & Sciences,Division of Social Sciences,Geography,Industry,60500,0,0,60500,True


In [14]:
non = sum(stem_industry.where('Non_Stem', True).column('Total'))
stem = sum(stem_industry.where('Non_Stem', False).column('Total'))
print(non, "to non-stem from industry")
print(stem, "to stem from industry")
print(stem/non ,": 1 stem to non-stem ratio")

1500620 to non-stem from industry
120635641 to stem from industry
80.3905325799 : 1 stem to non-stem ratio


In [35]:
# stem_industry.where('Non_Stem', False).where('Total', are.below(0)).show()
stem_industry.num_rows

448

### Other analysis on private funding

In [21]:
industry.where("Total", are.below(0))

Unnamed: 0,Sponsor,﻿Control Unit,Division,Department,Sponsor Class,2014,2015,2016,Total
30,Aditazz,Vice Chancellor - Research,Academic Research Units,CEDR Center for Environmental Design Research,Industry,0,-28000,0,-28000
165,"Applied Materials, Inc.",Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,0,-175000,0,-175000
207,"BP Group (BP America, BP Exploration, British Petroleum)",Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,-200000,0,0,-200000
496,Charles River Analytics,Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,0,-2,0,-2
926,Fairchild Semiconductor (Schlumberger),Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,0,0,-100000,-100000
981,"Gen9, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,-7000,0,0,-7000
996,"General Electric Company (incl GE Healthcare, GE Light Inst)",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,-200000,0,-200000
1028,GlobalFoundries U.S. INC.,Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,-160000,0,0,-160000
1105,"Hitachi, Ltd (incl Hitachi America, Ltd)",Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,0,-100000,0,-100000
1132,IBM International Business Machines Corporation,Colleges,College of Engineering,ERSO Engineering Research Support Organization,Industry,0,0,-1018333,-1018333
